# Atividade 3 - PCA


Entrega: 03/04/2020

ATIVIDADE: trabalhar o dataset nos aspectos:
    - Eliminação (ou não) de instâncias com missing values
    - Normalização
    - Balanceamento de classes

Trabalho:
- Objetivo: reduzir a dimensionalidade do dataset de vocês... (achar os componentes principais do dataset e decidir quantos componentes vai usar )

PDF de slides:
- Métodos: função utilizada e parâmetros (R, Matlab, python), se foi necessário algum pré-processamento dos dados, qual matriz utilizou (e se normalizou os dados ou não) e por quê…
– Pode testar mais de uma alternativa para comparar os resultados
- Resultados:
    * mostrar os PCs (coeficientes) e as variâncias acumuladas
    * com base nos resultados, decidir quantos componentes vai usar, explicando o porquê dessa escolha
- Discussão: sua análise crítica dos resultados scripts utilizados (só para postar no Moodle)

In [1]:
# imports

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# normalizacap
from sklearn import preprocessing

# Balanceamento das classes
from imblearn.combine import SMOTETomek

# Modelagem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

random_state=5007

## Import do Dataset

In [2]:
df = pd.read_csv('../data/kag_risk_factors_cervical_cancer.csv')

print(df.shape)

df.head()

(858, 36)


,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,IUD (years),STDs,STDs (number),STDs:condylomatosis,STDs:cervical condylomatosis,STDs:vaginal condylomatosis,STDs:vulvo-perineal condylomatosis,STDs:syphilis,STDs:pelvic inflammatory disease,STDs:genital herpes,STDs:molluscum contagiosum,STDs:AIDS,STDs:HIV,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0


## Pré-Processamento

In [3]:
# substituindo ? por NAN
df.replace('?', np.NAN,inplace=True)

In [4]:
# Transforma as feature em numericas
df_processed = df.apply(pd.to_numeric, errors="ignore")

# Resultado final com as devidas alteracoes
df_processed.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 36 columns):
Age                                   858 non-null int64
Number of sexual partners             832 non-null float64
First sexual intercourse              851 non-null float64
Num of pregnancies                    802 non-null float64
Smokes                                845 non-null float64
Smokes (years)                        845 non-null float64
Smokes (packs/year)                   845 non-null float64
Hormonal Contraceptives               750 non-null float64
Hormonal Contraceptives (years)       750 non-null float64
IUD                                   741 non-null float64
IUD (years)                           741 non-null float64
STDs                                  753 non-null float64
STDs (number)                         753 non-null float64
STDs:condylomatosis                   753 non-null float64
STDs:cervical condylomatosis          753 non-null float64
STDs:vaginal

## Eliminação (ou não) de instâncias com missing values

Para features continuas --> preenche com a mediana (para nao ter muito impacto com outliers)

Hipotese:
 - Para features categoricas --> preenche com o valor mais frequente
 
No decorrer dos experimentos validar se a hipotese é aceita ou não.

In [7]:
# features continuas
continuous_feat = ['Number of sexual partners', 'First sexual intercourse', 'Num of pregnancies', 
                   'Smokes', 'Smokes (years)', 'Smokes (packs/year)', 'Hormonal Contraceptives', 
                   'Hormonal Contraceptives (years)', 'IUD', 'IUD (years)', 'STDs', 'STDs (number)', 
                   'STDs:condylomatosis', 'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis', 
                   'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis', 
                   'STDs:pelvic inflammatory disease', 'STDs:genital herpes', 'STDs:molluscum contagiosum', 
                   'STDs:AIDS', 'STDs:HIV', 'STDs:Hepatitis B', 'STDs:HPV', 
                   'STDs: Time since first diagnosis','STDs: Time since last diagnosis'] 

# preenche com a mediana
for cont_col in continuous_feat:
    df_processed[cont_col] = df_processed[cont_col].fillna(df_processed[cont_col].median())
    
    
# preenche com o valor mais frequente
df_processed['Smokes'] = df_processed['Smokes'].fillna(0)
df_processed['Hormonal Contraceptives'] = df_processed['Hormonal Contraceptives'].fillna(1)
df_processed['IUD'] = df_processed['IUD'].fillna(0)
df_processed['STDs'] = df_processed['STDs'].fillna(0)

In [8]:
# salva dataset
df_processed.to_csv('../data/df_processed.csv')

## Normalização

A principio vamos testar o MinMax Scaler

In [9]:
# Aplica MinMax
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))

df_normalized = minmax_scale.fit_transform(df_processed)

In [10]:
# transforma de volta para dataframe
df_normalized = pd.DataFrame(df_normalized, columns = df_processed.columns.tolist())

df_normalized.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,IUD (years),STDs,STDs (number),STDs:condylomatosis,STDs:cervical condylomatosis,STDs:vaginal condylomatosis,STDs:vulvo-perineal condylomatosis,STDs:syphilis,STDs:pelvic inflammatory disease,STDs:genital herpes,STDs:molluscum contagiosum,STDs:AIDS,STDs:HIV,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,0.070423,0.111111,0.227273,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.028169,0.000000,0.181818,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.295775,0.000000,0.318182,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.549296,0.148148,0.272727,0.363636,1.0,1.0,1.0,1.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.095238,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.464789,0.074074,0.500000,0.363636,0.0,0.0,0.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# salva dataset
df_normalized.to_csv('../data/df_normalized.csv')

## Split em Train e Test

In [12]:
# Utilizando todas as features como preditoras
X = df_normalized.drop('Biopsy', axis=1)
y = df_normalized['Biopsy']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = 0.25, random_state=random_state)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((643, 35), (215, 35), (643,), (215,))

## Balanceamento de classes

Vamos aplicar a tecnica de sampling

a partir de amostras, vamos criar novos registros  nosso conjunto de forma que cada amostra represente o target de forma mais equilibrada

In [13]:
# documentacao SMOTETomek: 
# https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.combine.SMOTETomek.html

cc = SMOTETomek(random_state=random_state)
X_train_res, y_train_res = cc.fit_resample(X_train, y_train)

X_train_res.shape, y_train_res.shape

((1208, 35), (1208,))

Aumentamos nosso dataset de treino e teste de 643 para 1208

# PCA